## Planner-Executer architecture:

It is a type of the arcitecture where the system first of all computes and thinks of a sequence of task/step to perform to achieve the goal before even starting to execute the first step. It gets ready for battle without starting it. It first of all makes a breif sequence of step the system has to follow to achieve goalm where the steps are the simple sub-goals that are made from the main complex goal.

It is different from ReAct architecture as ReAct is for one step reasoning(step) as a time where as this is for plan once and reach the goal with the steps in the plan. ReAct is done for the goal that is lighter than the goal given to planner-executer as we dont have to reason more in ReaAct and ReAct is high bill generator as we have to keep reasoning as the goal gets complex which consumes high token eventually leading to high cost.

### Components of planner architecture:

It contains of two components:

1. planner: the one that makes the plan by decomposing the goal into sub-gials that are simple and can be executed with 1 or 2 tool calls
2. executor : the one that reads the plan step-by-step and executes the task and eventually reaches the goal.
